# Flappy Bird Machine Learning Using Reinforcement Algorithm

### Deep Q-learning using Flappy Bird Game
Flappy Bird was a popular mobile game originally developed by Vietnamese video game artist and programmer Dong Nguyen. In it, the player controls a bird and tries to fly between green pipes without hitting them.

<img scr="assets/screenshots/flappy_screenshot.png">